In [1]:
! . .venv/bin/activate
! sstat -j $SLURM_JOB_ID 

JobID         MaxVMSize  MaxVMSizeNode  MaxVMSizeTask  AveVMSize     MaxRSS MaxRSSNode MaxRSSTask     AveRSS MaxPages MaxPagesNode   MaxPagesTask   AvePages     MinCPU MinCPUNode MinCPUTask     AveCPU   NTasks AveCPUFreq ReqCPUFreqMin ReqCPUFreqMax ReqCPUFreqGov ConsumedEnergy  MaxDiskRead MaxDiskReadNode MaxDiskReadTask  AveDiskRead MaxDiskWrite MaxDiskWriteNode MaxDiskWriteTask AveDiskWrite TRESUsageInAve TRESUsageInMax TRESUsageInMaxNode TRESUsageInMaxTask TRESUsageInMin TRESUsageInMinNode TRESUsageInMinTask TRESUsageInTot TRESUsageOutAve TRESUsageOutMax TRESUsageOutMaxNode TRESUsageOutMaxTask TRESUsageOutMin TRESUsageOutMinNode TRESUsageOutMinTask TRESUsageOutTot 
------------ ---------- -------------- -------------- ---------- ---------- ---------- ---------- ---------- -------- ------------ -------------- ---------- ---------- ---------- ---------- ---------- -------- ---------- ------------- ------------- ------------- -------------- ------------ --------------- --------------- 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [16]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

In [3]:
project_dir = "../../../"
games = pd.read_csv(f'{project_dir}/games.csv')
plays = pd.read_csv(f'{project_dir}/plays.csv')
players = pd.read_csv(f'{project_dir}/players.csv')
tracking_df = pd.read_csv(f'{project_dir}/tracking_week_2.csv')
player_play = pd.read_csv(f'{project_dir}/player_play.csv')

In [4]:
# Filter for plays where any defensive player caused pressure
# Select only rows where 'causedPressure' is True, and keep unique gameId-playId pairs
plays_with_pressure = player_play[player_play['causedPressure'] == True][['gameId', 'playId']].drop_duplicates()

# Merge plays with plays_with_pressure to create a labeled dataset
# Add a new column 'causedPressure' to indicate if pressure was caused on a play
# Use a left merge and set '_merge' column to check if the play exists in plays_with_pressure
plays['causedPressure'] = plays.merge(plays_with_pressure, on=['gameId', 'playId'], how='left', indicator=True)['_merge'] == 'both'
plays['causedPressure'] = plays['causedPressure'].astype(int)  # Convert boolean values to 0/1 

# Drop duplicates to ensure unique plays
# Keep only one record per gameId-playId pair
dataset = plays.drop_duplicates(subset=['gameId', 'playId'])

# dataset200 = dataset.head(200) # save only 200 rows for viewing
# output_path = f'ml_friendly_dataset.csv'
# dataset200.to_csv(output_path, index=False)


# Feature 1: Offensive Formation

In [7]:
# Feature 1: offenseFormation
unique_offense_formations = dataset['offenseFormation'].unique()
print(unique_offense_formations)

# One hot encode the offensive formation
one_hot_encoded_offense_formations = pd.get_dummies(dataset['offenseFormation'], prefix='formation')
ml_dataset = pd.concat([dataset, one_hot_encoded_offense_formations], axis=1) 

# Print the new offensive formations
offense_cols = one_hot_encoded_offense_formations.columns
print(ml_dataset[offense_cols].head())



['EMPTY' 'SHOTGUN' 'PISTOL' 'SINGLEBACK' nan 'JUMBO' 'I_FORM' 'WILDCAT']
   formation_EMPTY  formation_I_FORM  formation_JUMBO  formation_PISTOL  \
0             True             False            False             False   
1             True             False            False             False   
2            False             False            False             False   
3            False             False            False             False   
4            False             False            False              True   

   formation_SHOTGUN  formation_SINGLEBACK  formation_WILDCAT  
0              False                 False              False  
1              False                 False              False  
2               True                 False              False  
3               True                 False              False  
4              False                 False              False  


# Feature 2: Defensive Formation

In [8]:
# One hot encode the defensive formation
one_hot_encoded_defense_formations = pd.get_dummies(dataset['pff_passCoverage'], prefix='formation')
ml_dataset = pd.concat([ml_dataset, one_hot_encoded_defense_formations], axis=1)

# Get the new defensive formations columns
defense_cols = one_hot_encoded_defense_formations.columns
print(ml_dataset[defense_cols].head())

   formation_2-Man  formation_Bracket  formation_Cover 6-Left  \
0            False              False                   False   
1            False              False                   False   
2            False              False                   False   
3            False              False                   False   
4            False              False                   False   

   formation_Cover-0  formation_Cover-1  formation_Cover-1 Double  \
0              False              False                     False   
1              False              False                     False   
2              False              False                     False   
3              False              False                     False   
4              False               True                     False   

   formation_Cover-2  formation_Cover-3  formation_Cover-3 Cloud Left  \
0              False               True                         False   
1              False              False         

# Feature 3: Down

# More ideas:
Quarter
Shift Since Lineset
Use them all and use feature selection